# About
 * This notebook uses **LLM agents** to post a job. 

#### a) LLM Agents
   * **seeker**: who gathers information about a position (e.g. `a plant scientist`)
   * **writer**: who writes a job post

    
#### b) Tasks
   * **searching**: the seeker performs industry analysis, collects information on hiring needs, and lists SEO keywords
   * **writing**: the writer uses the information provided and writes a job post

 
#### c) Technical Background
 * **`llama3-8b` LLM model** hosted on **Groq**. 
 * **[crewai](https://github.com/joaomdmoura/crewAI)**, a framework to build multi-agent applications.
 * The idea and most of the code were taken from [AI Crew for Jobs postings](https://github.com/joaomdmoura/crewAI-examples/tree/main/job-posting). Adjustments were made to the model and tools.



# 1. Settings

In [1]:
# install packages if needed
#%pip install crewai crewai_tools langchain_groq 

### Import packages

In [2]:
import os


# model
from langchain_groq import ChatGroq

# agent 
from crewai import Agent, Task, Crew

# agent tools
from crewai_tools.tools import EXASearchTool


# display results in Markdown format
from IPython.display import Markdown

# for using GROQ_API and EXA_API_KEY 
# save GROQ_API_KEY and EXA_API_KEY in a file `.env` then load_dotenv()
import dotenv
dotenv.load_dotenv()

True

### Provide input information about a position

In [3]:
company_description = "A Martian agriculture company"
hiring_needs = "A remote-working plant scientist focuses on overcoming potato production barriers on Mars"
specific_benefits = "Your work will have an impact. We offer a competitive salary with high quality health insurance"

# 2. Create agents

### Model

In [4]:
# llm model 
llm_model_id ="llama3-8b-8192"

# Use llm model from Groq 
# Get an API here: https://console.groq.com/keys
llm=ChatGroq(temperature=0,
             model_name=llm_model_id)

### Tools

In [5]:
# Initialize the tool for internet searching capabilities
# get an API here: https://dashboard.exa.ai/
exa_search_tool = EXASearchTool()


### Agent 

In [6]:
# define your agents with roles and goals

# role: searching information 
seeker = Agent(
    llm=llm,
    role='Research Analyst',
    goal='Analyze the company description to generate insights on company values and specific needs.',
    tools=[exa_search_tool],
    backstory='Expert in identifying key values and needs from various sources, including websites and brief descriptions.',
    verbose=True,
    allow_delegation=False)

# role: writing
writer = Agent(
    llm=llm,
    role='Job Description Writer',
    goal='Use insights from the Research Analyst to create a detailed, engaging, and enticing job posting.',
    backstory='Skilled in crafting compelling job descriptions that resonate with the company value and attract the right candidates.',
    verbose=True,
    allow_delegation=False)



### Tasks

In [7]:
searching = Task(
    description=(
        f"""
        1. Analyze the provided company description: {company_description}. 
          Conduct an in-depth analysis of the industry. 
          Investigate current trends, challenges, and opportunities within the industry, utilizing market reports, recent developments, and expert opinions.
          Assess how these factors could impact the role being hired for and the overall attractiveness of the position to potential candidates.
		  Consider how the company's position within this industry and its response to these trends could be leveraged to attract top talent. 
		  Compile a report summarizing these insights, specifically how they can be leveraged in a job posting to attract the right candidates.
          Include in your report how the role contributes to addressing industry challenges or seizing opportunities.
        2. Based on the hiring needs: {hiring_needs}, identify the key skills, experiences, and qualities the ideal candidate should possess for the role. 
          Consider the company's competitive landscape and industry trends. 
          Prepare a list of recommended job requirements and qualifications that align with the company's needs and values.
        3. Include SEO keywords"""
    ),
    expected_output="""A comprehensive report detailing the company's values and mission, along with specific selling points relevant to the job role.
    Include a list of recommended skills, experiences, and qualities for the ideal candidate, aligned with the company's values and the specific role's requirements.
    List SEO keywords.
    """,
    agent=seeker,
)

writing = Task(
    description=(
        f"""1. Draft a job posting for the role described by the hiring manager: {hiring_needs}.
          Develop a detailed content outline.
          Use the insights on {company_description} to start with a compelling introduction, 
          followed by a detailed role description, responsibilities, and required skills and qualifications. 
          Ensure the tone aligns with the company's values and incorporate any unique benefits or opportunities offered by the company.
		  Specfic benefits: {specific_benefits}
        2. Review the draft job posting for the role:{hiring_needs}. 
          Check for clarity, engagement, grammatical accuracy, and alignment with the company's values. 
          Edit and refine the content, ensuring it speaks directly to the desired candidates and accurately reflects the role's unique benefits and opportunities. 
        3. Incorporate SEO keywords naturally.     
        4. Sections/Subtitles are properly named in an engaging manner.
        5. Proofread for grammatical errors and add a catchy job post title   
      """
    ),
    expected_output="""
    A detailed, engaging, and polished job posting that includes an introduction, role description, responsibilities, requirements, and unique company benefits.
    The tone should resonate with the company's culture and values, aimed at attracting the right candidates.
    Formated in markdown.""",
    agent=writer,
)


### Assemble the crew and assign tasks

In [8]:
# instantiate your crew with a sequential process
crew = Crew(
  agents=[seeker, writer],
  tasks=[searching, writing],
  verbose=1,
)


### Kick off

In [9]:
# get your crew to work!
result = crew.kickoff()


 [2024-06-22 12:53:34][DEBUG]: == Working Agent: Research Analyst


> Entering new CrewAgentExecutor chain...
Thought: I need to start by analyzing the company description to generate insights on company values and specific needs.

Action: Search the internet

Action Input: {search_query: 'Martian agriculture company'}
I apologize for the mistake. Here's the corrected attempt:

Thought: I need to start by analyzing the company description to generate insights on company values and specific needs.

Action: Search the internet

Action Input: {'search_query': 'Martian agriculture company'}
 


Search results: Title: Precision AgTech for Insect Farms
Score: 0.19142910838127136
Url: https://bugmars.com/
ID: https://bugmars.com/
---
Title: Loading...
Score: 0.1863366663455963
Url: https://marsag.org/
ID: https://marsag.org/
---
Title: Official Website
Score: 0.1806725710630417
Url: https://www.promiseofmars.com/home
ID: https://www.promiseofmars.com/home
---
Title: Farms for Earth & Space | 

# 3. Result

In [10]:
Markdown(result)

**Job Title:** Remote-Working Plant Scientist: Overcoming Potato Production Barriers on Mars
============================================================

**Introduction**
---------------

Are you a plant scientist passionate about sustainable and innovative farming practices? Do you want to be part of a mission to feed the world through Martian agriculture? We're seeking a talented remote-working plant scientist to join our team and help overcome potato production barriers on Mars. As a key member of our team, you'll have the opportunity to work on cutting-edge projects, develop new skills, and make a real impact on the future of food production.

**Role Description**
-------------------

As a remote-working plant scientist, you'll be responsible for conducting research and development to overcome potato production barriers on Mars. You'll work closely with our team of experts to design and implement sustainable and innovative farming practices that improve crop yields and reduce waste. Your work will have a direct impact on the success of our mission to feed the world through Martian agriculture.

**Responsibilities**
-------------------

* Conduct research and development to overcome potato production barriers on Mars
* Design and implement sustainable and innovative farming practices
* Collaborate with our team of experts to develop new crop varieties and improve crop yields
* Analyze data and identify trends to inform decision-making
* Develop and maintain relationships with stakeholders, including farmers, suppliers, and customers
* Stay up-to-date with the latest research and developments in plant science and agriculture

**Requirements and Qualifications**
--------------------------------

* Bachelor's degree in plant science, agriculture, or a related field
* 2+ years of experience in plant science, agriculture, or a related field
* Strong understanding of plant biology and physiology
* Experience with plant breeding and genetics
* Strong analytical and problem-solving skills
* Ability to work independently and as part of a team
* Strong communication and interpersonal skills

**Unique Company Benefits**
---------------------------

* Competitive salary with high-quality health insurance
* Opportunity to work on cutting-edge projects and make a real impact on the future of food production
* Collaborative and dynamic work environment
* Flexible remote work arrangements
* Professional development opportunities

**How to Apply**
----------------

If you're passionate about plant science, agriculture, and making a difference in the world, we'd love to hear from you. Please submit your resume and a cover letter explaining why you're the perfect fit for this role.

**Contact Information**
----------------------

Email: [jobs@martianagriculture.com](mailto:jobs@martianagriculture.com)
Phone: 555-555-5555
Website: martianagriculture.com/careers

**SEO Keywords:** Martian agriculture, potato production, sustainable farming practices, innovative farming practices, feeding the world, plant science, agriculture, plant breeding, genetics, analytical skills, problem-solving skills, communication skills, interpersonal skills